# Support Vector Machine (SVM)
   
   
### 본 캐글 코드는 아래와 같은 순서로 작성됨.  
   
   
1. 데이터 전처리
2. default hyperparameter를 사용한 SVM 모델     
   2.1. default linear kernel   
   2.2. default RBF kernel   
   2.3. default polynomial kernel
3. 각 SVM 모델 별 parameter 최적화   
   3.1. linear kernel : C
   3.2. RBF kernel : C, gamma   
   3.3. polynomial kernel : C, gamma, degree   
   
________ 

* 위 파트 중, 3파트의 경우 sklearn.cross_validation, sklearn.grid_search를 load 해야 함.   
* 본 코드는 캐글 사이트에서 작성되었으나, 위의 두 모듈이 load 되지 않음.   
* 제대로 load 되었을 시 작성해야 하는 코드는 작성해 두었으니, 개인 python 환경에서 문제 없이 load 되면 실행해 보시면 좋을 것 같습니다!

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/voicegender/voice.csv


## 1. 데이터 전처리

In [3]:
# load library
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
# data load
df = pd.read_csv("/kaggle/input/voicegender/voice.csv")
df.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male


In [5]:
# 각 feature 간의 correlation 확인
df.corr()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx
meanfreq,1.000000,-0.739039,0.925445,0.911416,0.740997,-0.627605,-0.322327,-0.316036,-0.601203,-0.784332,0.687715,1.000000,0.460844,0.383937,0.274004,0.536666,0.229261,0.519528,0.515570,-0.216979
sd,-0.739039,1.000000,-0.562603,-0.846931,-0.161076,0.874660,0.314597,0.346241,0.716620,0.838086,-0.529150,-0.739039,-0.466281,-0.345609,-0.129662,-0.482726,-0.357667,-0.482278,-0.475999,0.122660
median,0.925445,-0.562603,1.000000,0.774922,0.731849,-0.477352,-0.257407,-0.243382,-0.502005,-0.661690,0.677433,0.925445,0.414909,0.337602,0.251328,0.455943,0.191169,0.438919,0.435621,-0.213298
Q25,0.911416,-0.846931,0.774922,1.000000,0.477140,-0.874189,-0.319475,-0.350182,-0.648126,-0.766875,0.591277,0.911416,0.545035,0.320994,0.199841,0.467403,0.302255,0.459683,0.454394,-0.141377
Q75,0.740997,-0.161076,0.731849,0.477140,1.000000,0.009636,-0.206339,-0.148881,-0.174905,-0.378198,0.486857,0.740997,0.155091,0.258002,0.285584,0.359181,-0.023750,0.335114,0.335648,-0.216475
IQR,-0.627605,0.874660,-0.477352,-0.874189,0.009636,1.000000,0.249497,0.316185,0.640813,0.663601,-0.403764,-0.627605,-0.534462,-0.222680,-0.069588,-0.333362,-0.357037,-0.337877,-0.331563,0.041252
skew,-0.322327,0.314597,-0.257407,-0.319475,-0.206339,0.249497,1.000000,0.977020,-0.195459,0.079694,-0.434859,-0.322327,-0.167668,-0.216954,-0.080861,-0.336848,-0.061608,-0.305651,-0.304640,-0.169325
kurt,-0.316036,0.346241,-0.243382,-0.350182,-0.148881,0.316185,0.977020,1.000000,-0.127644,0.109884,-0.406722,-0.316036,-0.194560,-0.203201,-0.045667,-0.303234,-0.103313,-0.274500,-0.272729,-0.205539
sp.ent,-0.601203,0.716620,-0.502005,-0.648126,-0.174905,0.640813,-0.195459,-0.127644,1.000000,0.866411,-0.325298,-0.601203,-0.513194,-0.305826,-0.120738,-0.293562,-0.294869,-0.324253,-0.319054,0.198074
sfm,-0.784332,0.838086,-0.661690,-0.766875,-0.378198,0.663601,0.079694,0.109884,0.866411,1.000000,-0.485913,-0.784332,-0.421066,-0.362100,-0.192369,-0.428442,-0.289593,-0.436649,-0.431580,0.211477


In [6]:
# null값 여부 확인
df.isnull().sum()

meanfreq    0
sd          0
median      0
Q25         0
Q75         0
IQR         0
skew        0
kurt        0
sp.ent      0
sfm         0
mode        0
centroid    0
meanfun     0
minfun      0
maxfun      0
meandom     0
mindom      0
maxdom      0
dfrange     0
modindx     0
label       0
dtype: int64

In [7]:
# separating features and labels
X = df.iloc[:,:-1]
X.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,0.000000,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,0.000000,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,0.000000,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,0.083878,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,0.104261,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274


In [8]:
# label 컬럼의 성별(문자형 컬럼)을 숫자형으로 변경
from sklearn.preprocessing import LabelEncoder
y = df.iloc[:,-1]

# male -> 1, female -> 0으로 변경

gender_encoder = LabelEncoder()
y = gender_encoder.fit_transform(y)
y

array([1, 1, 1, ..., 0, 0, 0])

* 데이터 표준화 (Data Standardisation)
  
데이터 표준화는 각 feature의 분포를 평균=0, 표준편차=1인 분포가 되도록 이동하는 것.    
만약 각 feature이 표준 정규 분포 형태를 이루지 않는다면, 적용 시 조심해야 함.

In [9]:
# 각 데이터 값을 -1과 1 사이로 변경
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [10]:
# train set과 test set으로 변경
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
#test set은 전체 데이터의 20%만 선택

## 2. default hyperparameter를 사용한 SVM 모델

* SVM (using default hyperparameter)

In [11]:
from sklearn.svm import SVC
from sklearn import metrics
svc = SVC()
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
print('Accuracy Score :')
print(metrics.accuracy_score(y_test,y_pred))

Accuracy Score :
0.9763406940063092


* SVM (using default linear kernel)

In [12]:
svc = SVC(kernel='linear')
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
print('Accuracy Score :')
print(metrics.accuracy_score(y_test,y_pred))

Accuracy Score :
0.9779179810725552


* SVM (using default RBF kernel)

In [13]:
svc = SVC(kernel='rbf')
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
print('Accuracy Score :')
print(metrics.accuracy_score(y_test,y_pred))

Accuracy Score :
0.9763406940063092


* SVM (using default polynomial kernel)

In [14]:
svc = SVC(kernel='poly')
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
print('Accuracy Score :')
print(metrics.accuracy_score(y_test,y_pred))
# 정확도 하락 (polynomial kernel은 training dataset을 overfitting해서 일 가능성 존재)

Accuracy Score :
0.9589905362776026


default kernel을 사용한 세 경우의 정확도는 다음과 같음.
* default linear kernel : 0.9779179810725552
* default RBF kernel : 0.9763406940063092
* default polynomial kernel : 0.9589905362776026
    
위의 세 경우의 정확도가 다르게 나온 이유는 전체 data set을 train / test set으로 나누는 방법 때문   
본 코드에서는 train / test set을 랜덤하게 나눔.   
따라서, 어떤 데이터들이 train set으로 들어가고, test set이 어떻게 분포되어있는가에 따라 정확도가 달라질 수 있음.

## 3. 각 SVM 모델 별 parameter 최적화

SVM에서 C 파라미터란, 얼마나 각 training 예제를 misclassifying 하지 않을 것인지 결정하는 SVM 파라미터를 의미함.   
C를 크게 설정하면, hyperplane이 모든 training point를 가장 잘 분류할 수 있는, smaller-margin을 선택하게 됨.   
C를 작게 설정하면, larger-margin이 선택되며, 이 경우에는 hyperplane이 point 중 여러 개를 잘못 분류하게 될 수 있음.   
   
따라서, C를 매우 크게 설정하면 모델의 과적합을 유발할 수 있으며, C를 매우 작게 설정하면 과소 적합을 유발할 수 있음.  
따라서, C 값은 주어진 데이터들을 일반화할 수 있는 값으로 설정되어야 함.  


In [16]:
from sklearn.cross_validation import cross_val_score
# 본 노트북은 캐글 사이트에서 작성되었음.
# 캐글에서 sklearn.cross_validation을 load하는데 문제가 생김.
# 개인의 python 설정 상에서 sklearn.cross_validation을 load할 수 있으면, 아래 코드를 통해 C 값에 따른 정확도를 확인할 수 있을 것임.

ModuleNotFoundError: No module named 'sklearn.cross_validation'

In [17]:
# C 값의 범의를 1부터 26으로 변화를 주며, 정확도 확인
C_range=list(range(1,26))
acc_score=[]
for c in C_range:
    svc = SVC(kernel='linear', C=c)
    scores = cross_val_score(svc, X, y, cv=10, scoring='accuracy')
    acc_score.append(scores.mean())
print(acc_score) 

NameError: name 'cross_val_score' is not defined

In [ ]:
# 위의 cross_val_score 관련 코드가 잘 실행되었다면,   
# 아래 그래프 그리는 코드를 통해 1~26의 C 값 중
# 어떤 값이 가장 좋은 정확도를 보이는지 확인할 수 있음.
# C 값은 linear, RBF, polynomial kernel 모두 값을 변화시켜가며
# 어떤 값이 가장 정확한 지 확인해볼 수 있음!


#import matplotlib.pyplot as plt
#%matplotlib inline
#
#C_values=list(range(1,26))
#
#plt.plot(C_values,acc_score)
#plt.xticks(np.arange(0,27,2))
#plt.xlabel('Value of C for SVC')
#plt.ylabel('Cross-Validated Accuracy')

Gamma parameter는 RBF kernel의 inverse of the standard deviation 임.   
즉, gamma parameter는 두 point 사이의 유사성을 의미함.   
Gamma 값을 작게 설정하면 큰 분산의 gaussian function이 가정됨.   
Gamma 값을 크게 설정하면 작은 분산의 gaussian function이 가정됨.

In [ ]:
# 본 코드도 cross_val_score load의 오류로 인해 본 노트북에서는 실행하지 않음.


#gamma_range=[0.0001,0.001,0.01,0.1,1,10,100]
#acc_score=[]
#for g in gamma_range:
#    svc = SVC(kernel='rbf', gamma=g)
#    scores = cross_val_score(svc, X, y, cv=10, scoring='accuracy')
#    acc_score.append(scores.mean())
#print(acc_score)  

In [ ]:
# 위에서 계산한 gamma 값에 따른 정확도를 가시화 할 수 있는 코드

#import matplotlib.pyplot as plt
#%matplotlib inline
#
#gamma_range=[0.0001,0.001,0.01,0.1,1,10,100]
#
#plt.plot(gamma_range,acc_score)
#plt.xlabel('Value of gamma for SVC ')
#plt.xticks(np.arange(0.0001,100,5))
#plt.ylabel('Cross-Validated Accuracy')

Degree 값에 따른 polynomial kernel

In [ ]:
#degree=[2,3,4,5,6]
#acc_score=[]
#for d in degree:
#    svc = SVC(kernel='poly', degree=d)
#    scores = cross_val_score(svc, X, y, cv=10, scoring='accuracy')
#    acc_score.append(scores.mean())
#print(acc_score) 

In [ ]:
#import matplotlib.pyplot as plt
#%matplotlib inline
#
#degree=[2,3,4,5,6]
#
#plt.plot(degree,acc_score,color='r')
#plt.xlabel('degrees for SVC ')
#plt.ylabel('Cross-Validated Accuracy')

아래의 코드는 linear, RBF, polynomial kernel에서 C, gamma, degree 파라미터의 최적 값을 찾기 위한 코드.  
본 코드는 sklearn.grid_search을 load해야 함.  
본 노트북에서는 sklearn.grid_search가 load 되지 않아 수행해보진 못하였으나, 해당 모듈이 load 된다면 수행해보길 바랍니다!

In [19]:
from sklearn.svm import SVC
svm_model= SVC()

In [20]:
tuned_parameters = {
 'C': (np.arange(0.1,1,0.1)) , 'kernel': ['linear'],
 'C': (np.arange(0.1,1,0.1)) , 'gamma': [0.01,0.02,0.03,0.04,0.05], 'kernel': ['rbf'],
 'degree': [2,3,4] ,'gamma':[0.01,0.02,0.03,0.04,0.05], 'C':(np.arange(0.1,1,0.1)) , 'kernel':['poly']
                   }

In [21]:
from sklearn.grid_search import GridSearchCV
model_svm = GridSearchCV(svm_model, tuned_parameters,cv=10,scoring='accuracy')

model_svm.fit(X_train, y_train)
print(model_svm.best_score_)

y_pred= model_svm.predict(X_test)
print(metrics.accuracy_score(y_pred,y_test))

ModuleNotFoundError: No module named 'sklearn.grid_search'